Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Set Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [3]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

Load Data

In [4]:
train_dataset = datasets.MNIST(root='./datatset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,)
test_dataset = datasets.MNIST(root='./datatset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True,)

Create Fully Connected Network

In [5]:
class MyNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=50),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=num_classes)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(x)

        return out


Initialize the Network

In [6]:
model = MyNN(input_size=input_size,num_classes=num_classes).to(device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the Network

In [8]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for batch_idx ,(data, target) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device=device)
        target = target.to(device=device)
        #Get to correct shape
        data = data.reshape(data.shape[0], -1)

        #forward
        scores = model(data)
        loss = criterion(scores, target)

        #Backward
        optimizer.zero_grad()
        loss.backward()

        #Gradient Descent or Adam Step
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, batch_idx+1, total_step, loss.item()))

Epoch [1/10], Step [1/938], Loss: 2.3390
Epoch [1/10], Step [2/938], Loss: 2.2986
Epoch [1/10], Step [3/938], Loss: 2.2631
Epoch [1/10], Step [4/938], Loss: 2.2356
Epoch [1/10], Step [5/938], Loss: 2.2740
Epoch [1/10], Step [6/938], Loss: 2.1778
Epoch [1/10], Step [7/938], Loss: 2.1037
Epoch [1/10], Step [8/938], Loss: 2.1013
Epoch [1/10], Step [9/938], Loss: 2.0526
Epoch [1/10], Step [10/938], Loss: 2.0512
Epoch [1/10], Step [11/938], Loss: 2.0196
Epoch [1/10], Step [12/938], Loss: 2.0187
Epoch [1/10], Step [13/938], Loss: 1.9485
Epoch [1/10], Step [14/938], Loss: 1.8963
Epoch [1/10], Step [15/938], Loss: 1.9385
Epoch [1/10], Step [16/938], Loss: 1.8453
Epoch [1/10], Step [17/938], Loss: 1.8957
Epoch [1/10], Step [18/938], Loss: 1.8471
Epoch [1/10], Step [19/938], Loss: 1.7765
Epoch [1/10], Step [20/938], Loss: 1.6973
Epoch [1/10], Step [21/938], Loss: 1.7202
Epoch [1/10], Step [22/938], Loss: 1.6749
Epoch [1/10], Step [23/938], Loss: 1.7121
Epoch [1/10], Step [24/938], Loss: 1.7027
E

Check Accuracy

In [9]:
def evaluate(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples=0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _,pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_samples += pred.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()

In [10]:
evaluate(train_loader, model)
evaluate(test_loader, model)

Checking accuracy on training data
Got 55875/60000 with accuracy 93.12
Checking accuracy on test data
Got 9269/10000 with accuracy 92.69
